## AOC 2024/12

### Part 1

--- Day 12: Garden Groups ---

Why not search for the Chief Historian near the gardener and his massive farm? There's plenty of food, so The Historians grab something to eat while they search.

You're about to settle near a complex arrangement of garden plots when some Elves ask if you can lend a hand. They'd like to set up fences around each region of garden plots, but they can't figure out how much fence they need to order or how much it will cost. They hand you a map (your puzzle input) of the garden plots.

Each garden plot grows only a single type of plant and is indicated by a single letter on your map. When multiple garden plots are growing the same type of plant and are touching (horizontally or vertically), they form a region. For example:

```
AAAA
BBCD
BBCC
EEEC
```

This 4x4 arrangement includes garden plots growing five different types of plants (labeled A, B, C, D, and E), each grouped into their own region.

In order to accurately calculate the cost of the fence around a single region, you need to know that region's area and perimeter.

The area of a region is simply the number of garden plots the region contains. The above map's type A, B, and C plants are each in a region of area 4. The type E plants are in a region of area 3; the type D plants are in a region of area 1.

Each garden plot is a square and so has four sides. The perimeter of a region is the number of sides of garden plots in the region that do not touch another garden plot in the same region. The type A and C plants are each in a region with perimeter 10. The type B and E plants are each in a region with perimeter 8. The lone D plot forms its own region with perimeter 4.

Visually indicating the sides of plots in each region that contribute to the perimeter using - and |, the above map's regions' perimeters are measured as follows:

```
+-+-+-+-+
|A A A A|
+-+-+-+-+     +-+
              |D|
+-+-+   +-+   +-+
|B B|   |C|
+   +   + +-+
|B B|   |C C|
+-+-+   +-+ +
          |C|
+-+-+-+   +-+
|E E E|
+-+-+-+
```

Plants of the same type can appear in multiple separate regions, and regions can even appear within other regions. For example:

```
OOOOO
OXOXO
OOOOO
OXOXO
OOOOO
```

The above map contains five regions, one containing all of the O garden plots, and the other four each containing a single X plot.

The four X regions each have area 1 and perimeter 4. The region containing 21 type O plants is more complicated; in addition to its outer edge contributing a perimeter of 20, its boundary with each X region contributes an additional 4 to its perimeter, for a total perimeter of 36.

Due to "modern" business practices, the price of fence required for a region is found by multiplying that region's area by its perimeter. The total price of fencing all regions on a map is found by adding together the price of fence for every region on the map.

In the first example, region A has price 4 * 10 = 40, region B has price 4 * 8 = 32, region C has price 4 * 10 = 40, region D has price 1 * 4 = 4, and region E has price 3 * 8 = 24. So, the total price for the first example is 140.

In the second example, the region with all of the O plants has price 21 * 36 = 756, and each of the four smaller X regions has price 1 * 4 = 4, for a total price of 772 (756 + 4 + 4 + 4 + 4).

Here's a larger example:

```
RRRRIICCFF
RRRRIICCCF
VVRRRCCFFF
VVRCCCJFFF
VVVVCJJCFE
VVIVCCJJEE
VVIIICJJEE
MIIIIIJJEE
MIIISIJEEE
MMMISSJEEE
```

It contains:

- A region of R plants with price 12 * 18 = 216.
- A region of I plants with price 4 * 8 = 32.
- A region of C plants with price 14 * 28 = 392.
- A region of F plants with price 10 * 18 = 180.
- A region of V plants with price 13 * 20 = 260.
- A region of J plants with price 11 * 20 = 220.
- A region of C plants with price 1 * 4 = 4.
- A region of E plants with price 13 * 18 = 234.
- A region of I plants with price 14 * 22 = 308.
- A region of M plants with price 5 * 12 = 60.
- A region of S plants with price 3 * 8 = 24.

So, it has a total price of `1930`.

What is the total price of fencing all regions on your map?

**AI Prompt**

I'm a skilled python developer, please help go consider various approaches for solving this problem and their pros/cons. I prefer simpler and understandable solutions, easily readable also for other developers. It's ok to generate code with testcases without explanation. I'll ask if I do not understand it. Always wrap test code into a function, to not clutter the global variable namespace.

**AI Response**

For this problem, I see a few main approaches we could take:

1. Graph-based approach using networkx:
   - Create a graph where each plot is a node
   - Connect plots of same type
   - Find connected components
   - For each component, calculate area (node count) and perimeter
   Pros: Clean abstraction, handles complex shapes well
   Cons: Might be overkill for simple grids

2. Flood fill / DFS approach:
   - Scan grid to find unvisited plots
   - For each, flood fill to find the region
   - Track visited cells and boundary counts
   Pros: Intuitive, good performance
   Cons: Need careful boundary checking

3. Union-find approach:
   - Join adjacent same-type plots
   - Calculate regions and boundaries at the end
   Pros: Very efficient for large grids
   Cons: More complex implementation

Would you like me to implement a solution using any of these approaches? Or would you prefer to explore a different strategy?

In [ ]:
sample_input = """
RRRRIICCFF
RRRRIICCCF
VVRRRCCFFF
VVRCCCJFFF
VVVVCJJCFE
VVIVCCJJEE
VVIIICJJEE
MIIIIIJJEE
MIIISIJEEE
MMMISSJEEE
""".strip()

**AI Prompt**

implement the function `find_regions` using the flood fill algorithm. Given a numpy 2D-array with platns (characters) return a `list[tuple[str, np.array]]`
- first item in tuple is plant name (char)
- secont item in tuple is region where that plant grows. Cell where plant grows is marked with True, cells where the plant does not grow are masked out with False
- if a plant grows in more unconnected regions, each of them is returned as a separated item in the result list
- For example, for `sample_input ` will the returned list contain for plant `I` these two numpy arrays (True depicted as `X`, False as `.`):

   ```
    ....XX....
    ....XX....
    ..........
    ..........
    ..........
    ..........
    ..........
    ..........
    ..........
    ..........
   ```
   
   and 

   ```
    ..........
    ..........
    ..........
    ..........
    ..........
    ..X.......
    ..XXX.....
    .XXXXX....
    .XXX.X....
    ...X......
   ```

**AI Response**

Here's an implementation of `find_regions` using flood fill approach:

```python
import numpy as np
from typing import List, Tuple

def find_regions(grid: np.ndarray) -> List[Tuple[str, np.ndarray]]:
    def flood_fill(x: int, y: int, plant: str) -> np.ndarray:
        if (x < 0 or x >= grid.shape[0] or y < 0 or y >= grid.shape[1] 
            or grid[x,y] != plant or visited[x,y]):
            return
        
        visited[x,y] = True
        region[x,y] = True
        
        for dx, dy in [(0,1), (1,0), (0,-1), (-1,0)]:
            flood_fill(x + dx, y + dy, plant)
    
    regions = []
    visited = np.zeros_like(grid, dtype=bool)
    
    for x in range(grid.shape[0]):
        for y in range(grid.shape[1]):
            if not visited[x,y]:
                plant = grid[x,y]
                region = np.zeros_like(grid, dtype=bool)
                flood_fill(x, y, plant)
                if region.any():  # if any True values in region
                    regions.append((plant, region.copy()))
    
    return regions

def test_find_regions():
    # Convert string input to numpy array
    grid = np.array([list(line) for line in sample_input.split('\n')])
    
    regions = find_regions(grid)
    
    # Print all regions for verification
    for i, (plant, region) in enumerate(regions):
        print(f"\nRegion {i+1}, Plant: {plant}")
        for row in region:
            print(''.join('X' if cell else '.' for cell in row))
        
    # Verify specific cases
    i_regions = [r for p, r in regions if p == 'I']
    assert len(i_regions) == 2, "Should find exactly 2 I regions"
    
    # Verify total number of regions
    assert len(regions) == 11, "Should find 11 total regions"
    
    print("All tests passed!")

test_find_regions()
```

This implementation:
1. Uses recursive flood fill
2. Keeps track of visited cells to avoid revisiting
3. Creates a boolean mask for each region
4. Returns list of tuples with plant type and region mask

Would you like me to explain any part in more detail?

In [ ]:
sample_grid = np.array([list(line) for line in sample_input.split('\n')])
sample_grid

array([['R', 'R', 'R', 'R', 'I', 'I', 'C', 'C', 'F', 'F'],
       ['R', 'R', 'R', 'R', 'I', 'I', 'C', 'C', 'C', 'F'],
       ['V', 'V', 'R', 'R', 'R', 'C', 'C', 'F', 'F', 'F'],
       ['V', 'V', 'R', 'C', 'C', 'C', 'J', 'F', 'F', 'F'],
       ['V', 'V', 'V', 'V', 'C', 'J', 'J', 'C', 'F', 'E'],
       ['V', 'V', 'I', 'V', 'C', 'C', 'J', 'J', 'E', 'E'],
       ['V', 'V', 'I', 'I', 'I', 'C', 'J', 'J', 'E', 'E'],
       ['M', 'I', 'I', 'I', 'I', 'I', 'J', 'J', 'E', 'E'],
       ['M', 'I', 'I', 'I', 'S', 'I', 'J', 'E', 'E', 'E'],
       ['M', 'M', 'M', 'I', 'S', 'S', 'J', 'E', 'E', 'E']], dtype='<U1')

In [ ]:
def visualize_grid(region: np.array):
    ret = []
    for row in region:
        ret.append(''.join('X' if cell else '.' for cell in row))
    print('\n'.join(ret))

In [ ]:
import numpy as np
from typing import List, Tuple

def find_regions(grid: np.ndarray) -> List[Tuple[str, np.ndarray]]:
    def flood_fill(x: int, y: int, plant: str) -> np.ndarray:
        if (x < 0 or x >= grid.shape[0] or y < 0 or y >= grid.shape[1] 
            or grid[x,y] != plant or visited[x,y]):
            return
        
        visited[x,y] = True
        region[x,y] = True
        
        for dx, dy in [(0,1), (1,0), (0,-1), (-1,0)]:
            flood_fill(x + dx, y + dy, plant)
    
    regions = []
    visited = np.zeros_like(grid, dtype=bool)
    
    for x in range(grid.shape[0]):
        for y in range(grid.shape[1]):
            if not visited[x,y]:
                plant = grid[x,y]
                region = np.zeros_like(grid, dtype=bool)
                flood_fill(x, y, plant)
                if region.any():  # if any True values in region
                    regions.append((plant, region.copy()))
    
    return regions

def test_find_regions():
    # Convert string input to numpy array
    grid = sample_grid
    
    regions = find_regions(grid)

    # Verify specific cases
    i_regions = [r for p, r in regions if p == 'I']
    assert len(i_regions) == 2, "Should find exactly 2 I regions"
    
    # Verify total number of regions
    assert len(regions) == 11, "Should find 11 total regions"
    
    print("All tests passed!")

test_find_regions()

All tests passed!


**AI Prompt**

write function `crop_grid` given a numpy boolean 2d array returns 2d array where
 - all rows or columns are rempved if all values are False

**AI Response**

Here's a function to crop a boolean grid by removing all-False rows and columns:

```python
def crop_grid(grid: np.ndarray) -> np.ndarray:
    # Find rows and columns that have at least one True
    rows_to_keep = grid.any(axis=1)
    cols_to_keep = grid.any(axis=0)
    
    # Return cropped array
    return grid[rows_to_keep][:, cols_to_keep]

def test_crop_grid():
    # Test case 1: Simple grid with empty borders
    test_grid = np.array([
        [False, False, False, False],
        [False, True, True, False],
        [False, True, True, False],
        [False, False, False, False]
    ])
    result = crop_grid(test_grid)
    assert result.shape == (2, 2)
    assert (result == True).all()
    
    # Test case 2: No cropping needed
    test_grid = np.array([[True]])
    result = crop_grid(test_grid)
    assert result.shape == (1, 1)
    
    print("All tests passed!")

test_crop_grid()
```

Would you like me to explain any part of this implementation?

In [ ]:
def crop_grid(grid: np.ndarray) -> np.ndarray:
    # Find rows and columns that have at least one True
    rows_to_keep = grid.any(axis=1)
    cols_to_keep = grid.any(axis=0)
    
    # Return cropped array
    return grid[rows_to_keep][:, cols_to_keep]

def test_crop_grid():
    # Test case 1: Simple grid with empty borders
    test_grid = np.array([
        [False, False, False, False],
        [False, True, True, False],
        [False, True, True, False],
        [False, False, False, False]
    ])
    result = crop_grid(test_grid)
    assert result.shape == (2, 2)
    assert (result == True).all()
    
    # Test case 2: No cropping needed
    test_grid = np.array([[True]])
    result = crop_grid(test_grid)
    assert result.shape == (1, 1)
    
    print("All tests passed!")

test_crop_grid()

All tests passed!


In [ ]:
def draw_sample_regions():
    regions = [(c,crop_grid(r)) for c,r in find_regions(sample_grid)]
    # Print all regions for verification
    for i, (plant, region) in enumerate(regions):
        print(f"\nRegion {i+1}, Plant: {plant}")
        visualize_grid(region)

draw_sample_regions()


Region 1, Plant: R
XXXX.
XXXX.
..XXX
..X..

Region 2, Plant: I
XX
XX

Region 3, Plant: C
...XX.
...XXX
..XX..
XXX...
.X....
.XX...
..X...

Region 4, Plant: F
.XX
..X
XXX
XXX
.X.

Region 5, Plant: V
XX..
XX..
XXXX
XX.X
XX..

Region 6, Plant: J
.X.
XX.
.XX
.XX
.XX
.X.
.X.

Region 7, Plant: C
X

Region 8, Plant: E
..X
.XX
.XX
.XX
XXX
XXX

Region 9, Plant: I
.X...
.XXX.
XXXXX
XXX.X
..X..

Region 10, Plant: M
X..
X..
XXX

Region 11, Plant: S
X.
XX


**AI Prompt**

load full data

**AI Response**

Here's how we can load the data from Advent of Code using the `aocd` library:

```python
from aocd import get_data
data = get_data(day=12, year=2024)
grid = np.array([list(line) for line in data.split('\n')])
print(f"Grid shape: {grid.shape}")
```

Would you like me to help you test the `find_regions` function on this data?

In [ ]:
from aocd import get_data
full_input = get_data(day=12, year=2024)
full_grid = np.array([list(line) for line in full_input.split('\n')])

In [ ]:
from random import randint

def draw_full_regions_sample(N=3):
    full_regions = [(c,crop_grid(r)) for c,r in find_regions(full_grid)]    
    for i in range(N):
        idx = randint(0, len(full_regions))
        print(f"\nRegion {idx}, Plant: {full_regions[idx][0]}")
        visualize_grid(full_regions[idx][1])

draw_full_regions_sample()


Region 356, Plant: G
........X..........
........X..........
........X..........
.....XXXXX.........
..XXXXXXXXXXXX.....
...XXXXXXXXXXXX....
...XXXXXXXXXXX.....
...XXXXXXXXXXXXXX..
...XXXXXXXXXXXXX...
...XXXXX...XXXXX...
.XXXXXXX....X..X...
.XX.XXXXX......XX..
.X..XX.........XX..
XX.............XX..
XX.............XX..
...............XX..
...............XX..
................XX.
................XX.
.................XX
.................XX

Region 392, Plant: N
X
X

Region 13, Plant: H
X


**AI Prompt**

lets compute region perimeter

**AI Response**

Let's create a function to compute the perimeter of a region. For each True cell in the region, we'll count how many of its edges are on the boundary (either touching a False cell or the edge of the grid).

```python
def compute_perimeter(region: np.ndarray) -> int:
    rows, cols = region.shape
    perimeter = 0
    
    for r in range(rows):
        for c in range(cols):
            if region[r, c]:
                # Check all 4 directions
                for dr, dc in [(0,1), (1,0), (0,-1), (-1,0)]:
                    nr, nc = r + dr, c + dc
                    # Count edge if neighbor is outside grid or False
                    if (nr < 0 or nr >= rows or 
                        nc < 0 or nc >= cols or 
                        not region[nr, nc]):
                        perimeter += 1
    return perimeter

def test_perimeter():
    # Test simple square
    square = np.array([
        [True, True],
        [True, True]
    ])
    assert compute_perimeter(square) == 8
    
    # Test single cell
    single = np.array([[True]])
    assert compute_perimeter(single) == 4
    
    print("Tests passed!")

test_perimeter()
```

Would you like me to add more test cases or explain how this works?

In [ ]:
def compute_perimeter(region: np.ndarray) -> int:
    rows, cols = region.shape
    perimeter = 0
    
    for r in range(rows):
        for c in range(cols):
            if region[r, c]:
                # Check all 4 directions
                for dr, dc in [(0,1), (1,0), (0,-1), (-1,0)]:
                    nr, nc = r + dr, c + dc
                    # Count edge if neighbor is outside grid or False
                    if (nr < 0 or nr >= rows or 
                        nc < 0 or nc >= cols or 
                        not region[nr, nc]):
                        perimeter += 1
    return perimeter

def test_perimeter():
    # Test simple square
    square = np.array([
        [True, True],
        [True, True]
    ])
    assert compute_perimeter(square) == 8
    
    # Test single cell
    single = np.array([[True]])
    assert compute_perimeter(single) == 4
    
    print("Tests passed!")

test_perimeter()

Tests passed!


**AI Prompt**

lets compute region area

**AI Response**

The area of a region is simply the count of True cells in the boolean array. Here's a simple function for that:

```python
def compute_area(region: np.ndarray) -> int:
    return np.sum(region)

def test_area():
    # Test simple square
    square = np.array([
        [True, True],
        [True, True]
    ])
    assert compute_area(square) == 4
    
    # Test single cell
    single = np.array([[True]])
    assert compute_area(single) == 1
    
    print("Tests passed!")

test_area()
```

Would you like me to add more test cases or proceed with computing the total price (area × perimeter) for each region?

In [ ]:
def compute_area(region: np.ndarray) -> int:
    return np.sum(region)

def test_area():
    # Test simple square
    square = np.array([
        [True, True],
        [True, True]
    ])
    assert compute_area(square) == 4
    
    # Test single cell
    single = np.array([[True]])
    assert compute_area(single) == 1
    
    print("Tests passed!")

test_area()

Tests passed!


In [ ]:
def test_area_perimeter_full_region_356(ridx=356):
    full_regions = [(c,crop_grid(r)) for c,r in find_regions(full_grid)]    
    plant, region = full_regions[ridx]
    visualize_grid(region)  

    assert compute_area(region) == 123

    # 23 top + 23 bottom + 30 right + 30 left
    assert compute_perimeter(region) == 23 + 23 + 30 + 30
    print("all tests passed")
    

test_perimeter_full_region_356()

........X..........
........X..........
........X..........
.....XXXXX.........
..XXXXXXXXXXXX.....
...XXXXXXXXXXXX....
...XXXXXXXXXXX.....
...XXXXXXXXXXXXXX..
...XXXXXXXXXXXXX...
...XXXXX...XXXXX...
.XXXXXXX....X..X...
.XX.XXXXX......XX..
.X..XX.........XX..
XX.............XX..
XX.............XX..
...............XX..
...............XX..
................XX.
................XX.
.................XX
.................XX
all tests passed


**AI Prompt**

write function `total_price` given result of `find_regions` as input
 - apply crop_grid for speedup
 - then test on sample_grid

**AI Response**

Here's the implementation of `total_price` function:

```python
def total_price(regions: List[Tuple[str, np.ndarray]]) -> int:
    total = 0
    for plant, region in regions:
        cropped = crop_grid(region)
        area = compute_area(cropped)
        perimeter = compute_perimeter(cropped)
        total += area * perimeter
    return total

def test_total_price():
    regions = find_regions(sample_grid)
    price = total_price(regions)
    assert price == 1930, f"Expected 1930, got {price}"
    
    # Let's also print details for verification
    for plant, region in regions:
        cropped = crop_grid(region)
        area = compute_area(cropped)
        perimeter = compute_perimeter(cropped)
        print(f"Plant {plant}: area={area} perimeter={perimeter} price={area*perimeter}")
    
    print("Test passed!")

test_total_price()
```

Would you like to run this on the full input now?

In [ ]:
def total_price(regions: List[Tuple[str, np.ndarray]]) -> int:
    total = 0
    for plant, region in regions:
        cropped = crop_grid(region)
        area = compute_area(cropped)
        perimeter = compute_perimeter(cropped)
        total += area * perimeter
    return total

def test_total_price():
    regions = find_regions(sample_grid)
    price = total_price(regions)
    assert price == 1930, f"Expected 1930, got {price}"
    
    # Let's also print details for verification
    for plant, region in regions:
        cropped = crop_grid(region)
        area = compute_area(cropped)
        perimeter = compute_perimeter(cropped)
        print(f"Plant {plant}: area={area} perimeter={perimeter} price={area*perimeter}")
    
    print("Test passed!")

test_total_price()

Plant R: area=12 perimeter=18 price=216
Plant I: area=4 perimeter=8 price=32
Plant C: area=14 perimeter=28 price=392
Plant F: area=10 perimeter=18 price=180
Plant V: area=13 perimeter=20 price=260
Plant J: area=11 perimeter=20 price=220
Plant C: area=1 perimeter=4 price=4
Plant E: area=13 perimeter=18 price=234
Plant I: area=14 perimeter=22 price=308
Plant M: area=5 perimeter=12 price=60
Plant S: area=3 perimeter=8 price=24
Test passed!


In [ ]:
def solve_part1():
    full_regions = find_regions(full_grid)
    return total_price(full_regions)
solve_part1()

1370100

this was the correct answer. lets proceed to part2

## Part2

--- Part Two ---

Fortunately, the Elves are trying to order so much fence that they qualify for a bulk discount!

Under the bulk discount, instead of using the perimeter to calculate the price, you need to use the number of sides each region has. Each straight section of fence counts as a side, regardless of how long it is.

Consider this example again:

```
AAAA
BBCD
BBCC
EEEC
```

The region containing type A plants has 4 sides, as does each of the regions containing plants of type B, D, and E. However, the more complex region containing the plants of type C has 8 sides!

Using the new method of calculating the per-region price by multiplying the region's area by its number of sides, regions A through E have prices 16, 16, 32, 4, and 12, respectively, for a total price of 80.

The second example above (full of type X and O plants) would have a total price of 436.

Here's a map that includes an E-shaped region full of type E plants:

```
EEEEE
EXXXX
EEEEE
EXXXX
EEEEE
```

The E-shaped region has an area of 17 and 12 sides for a price of 204. Including the two regions full of type X plants, this map has a total price of 236.

This map has a total price of 368:

```
AAAAAA
AAABBA
AAABBA
ABBAAA
ABBAAA
AAAAAA
```

It includes two regions full of type B plants (each with 4 sides) and a single region full of type A plants (with 4 sides on the outside and 8 more sides on the inside, a total of 12 sides). Be especially careful when counting the fence around regions like the one full of type A plants; in particular, each section of fence has an in-side and an out-side, so the fence does not connect across the middle of the region (where the two B regions touch diagonally). (The Elves would have used the Möbius Fencing Company instead, but their contract terms were too one-sided.)

The larger example from before now has the following updated prices:

- A region of R plants with price 12 * 10 = 120.
- A region of I plants with price 4 * 4 = 16.
- A region of C plants with price 14 * 22 = 308.
- A region of F plants with price 10 * 12 = 120.
- A region of V plants with price 13 * 10 = 130.
- A region of J plants with price 11 * 12 = 132.
- A region of C plants with price 1 * 4 = 4.
- A region of E plants with price 13 * 8 = 104.
- A region of I plants with price 14 * 16 = 224.
- A region of M plants with price 5 * 6 = 30.
- A region of S plants with price 3 * 6 = 18.

Adding these together produces its new total price of 1206.

What is the new total price of fencing all regions on your map?

**AI Prompt**

lets compute number of sides for region

**AI Response**

For computing the number of sides, we need to count each distinct straight section of the fence, both external and internal. Here's a proposed implementation:

```python
def compute_sides(region: np.ndarray) -> int:
    rows, cols = region.shape
    sides = set()  # store unique sides as (r1,c1,r2,c2) where r1,c1 is always smaller lexicographically
    
    for r in range(rows):
        for c in range(cols):
            if region[r, c]:
                # Check right and bottom neighbors
                for dr, dc in [(0,1), (1,0)]:
                    nr, nc = r + dr, c + dc
                    # If neighbor is outside or False, we have a side
                    if (nr >= rows or nc >= cols or not region[nr, nc]):
                        # Add side in canonical form (smaller coordinates first)
                        if dr == 0:  # vertical side
                            sides.add((r,c,r+1,c))
                        else:  # horizontal side
                            sides.add((r,c,r,c+1))
                    
                # Check left and top neighbors too
                for dr, dc in [(0,-1), (-1,0)]:
                    nr, nc = r + dr, c + dc
                    if (nr < 0 or nc < 0 or not region[nr, nc]):
                        if dr == 0:  # vertical side
                            sides.add((r,nc,r+1,nc))
                        else:  # horizontal side
                            sides.add((nr,c,nr,c+1))
    
    return len(sides)

def test_sides():
    # Test simple square (4 sides)
    square = np.array([
        [True, True],
        [True, True]
    ])
    assert compute_sides(square) == 4, f"Expected 4 sides for square, got {compute_sides(square)}"
    
    # Test single cell (4 sides)
    single = np.array([[True]])
    assert compute_sides(single) == 4, f"Expected 4 sides for single cell, got {compute_sides(single)}"
    
    # Test L shape (6 sides)
    l_shape = np.array([
        [True, False],
        [True, True]
    ])
    assert compute_sides(l_shape) == 6, f"Expected 6 sides for L-shape, got {compute_sides(l_shape)}"
    
    print("Tests passed!")

test_sides()
```

Would you like me to explain how this works in detail, or should we test it against the examples from Part 2?

**AI Prompt**

this is incorrect implementation. It fails already on sample inputs.

Let me guide you. please write function `perimiter_side_cells` then given `region` as np array and a `side`
  - side is one of `'top', 'left', 'right', 'bottom'
  - returns boolean np array, with masked cells  
  - cell set to True if it is part of perimeter for that side 
  - for example, on input

    ```
      ...XX..
      ...XX..
      .XXX...
      .XXXXX.
      .XX.XX.
      .XXXXX.
    ```  
    
    return

    ```
      ....X..
      ....X..
      ...X...
      .....X.
      ..X..X.
      .....X.
    ```

**AI Response**

Here's an implementation of `perimeter_side_cells` function:

```python
def perimeter_side_cells(region: np.ndarray, side: str) -> np.ndarray:
    result = np.zeros_like(region)
    rows, cols = region.shape
    
    for r in range(rows):
        for c in range(cols):
            if not region[r, c]:
                continue
                
            if side == 'right':
                # rightmost cell in its row or has empty cell to the right
                if c == cols-1 or not region[r, c+1]:
                    result[r, c] = True
            elif side == 'left':
                # leftmost cell in its row or has empty cell to the left
                if c == 0 or not region[r, c-1]:
                    result[r, c] = True
            elif side == 'top':
                # topmost cell in its column or has empty cell above
                if r == 0 or not region[r-1, c]:
                    result[r, c] = True
            elif side == 'bottom':
                # bottommost cell in its column or has empty cell below
                if r == rows-1 or not region[r+1, c]:
                    result[r, c] = True
    
    return result

def test_perimeter_side_cells():
    region = np.array([
        [0,0,0,1,1,0,0],
        [0,0,0,1,1,0,0],
        [0,1,1,1,0,0,0],
        [0,1,1,1,1,1,0],
        [0,1,1,0,1,1,0],
        [0,1,1,1,1,1,0]
    ], dtype=bool)
    
    print("Original region:")
    visualize_grid(region)
    
    for side in ['right', 'left', 'top', 'bottom']:
        print(f"\n{side.capitalize()} perimeter:")
        result = perimeter_side_cells(region, side)
        visualize_grid(result)

test_perimeter_side_cells()
```

Would you like me to explain how this works, or shall we proceed with using this function to count the sides correctly?

In [ ]:
def perimeter_side_cells(region: np.ndarray, side: str) -> np.ndarray:
    result = np.zeros_like(region)
    rows, cols = region.shape
    
    for r in range(rows):
        for c in range(cols):
            if not region[r, c]:
                continue
                
            if side == 'right':
                # rightmost cell in its row or has empty cell to the right
                if c == cols-1 or not region[r, c+1]:
                    result[r, c] = True
            elif side == 'left':
                # leftmost cell in its row or has empty cell to the left
                if c == 0 or not region[r, c-1]:
                    result[r, c] = True
            elif side == 'top':
                # topmost cell in its column or has empty cell above
                if r == 0 or not region[r-1, c]:
                    result[r, c] = True
            elif side == 'bottom':
                # bottommost cell in its column or has empty cell below
                if r == rows-1 or not region[r+1, c]:
                    result[r, c] = True
    
    return result

def test_perimeter_side_cells():
    region = np.array([
        [0,0,0,1,1,0,0],
        [0,0,0,1,1,0,0],
        [0,1,1,1,0,0,0],
        [0,1,1,1,1,1,0],
        [0,1,1,0,1,1,0],
        [0,1,1,1,1,1,0]
    ], dtype=bool)
    
    print("Original region:")
    visualize_grid(region)
    
    for side in ['right', 'left', 'top', 'bottom']:
        print(f"\n{side.capitalize()} perimeter:")
        result = perimeter_side_cells(region, side)
        visualize_grid(result)

test_perimeter_side_cells()

Original region:
...XX..
...XX..
.XXX...
.XXXXX.
.XX.XX.
.XXXXX.

Right perimeter:
....X..
....X..
...X...
.....X.
..X..X.
.....X.

Left perimeter:
...X...
...X...
.X.....
.X.....
.X..X..
.X.....

Top perimeter:
...XX..
.......
.XX....
....XX.
.......
...X...

Bottom perimeter:
.......
....X..
.......
...X...
.......
.XXXXX.


**AI Prompt**

write function `perimeter_num_sides` given a `region` and a `side`
 - call `perimeter_side_cells` 
 - count sequences of `True (X)` cells in the result
 - over all rows (for top, bottom sides)
 - over all columns (for right, left sides)
 - for example:
    - right and 

        ```
        ....X..
        ....X..
        ...X...
        .....X.
        ..X..X.
        .....X.    
        ```
        returns 4

    - `bottom` and 
        ```
        .......
        ....X..
        .......
        ...X...
        .......
        .XXXXX.        
        ```
        returns 3

**AI Response**

Here's an implementation of `perimeter_num_sides`:

```python
def perimeter_num_sides(region: np.ndarray, side: str) -> int:
    cells = perimeter_side_cells(region, side)
    sides = 0
    
    if side in ['top', 'bottom']:
        # Count sequences in each row
        for row in cells:
            in_sequence = False
            for cell in row:
                if cell and not in_sequence:
                    sides += 1
                    in_sequence = True
                elif not cell:
                    in_sequence = False
    else:  # left, right
        # Count sequences in each column
        for col in cells.T:  # transpose to iterate over columns
            in_sequence = False
            for cell in col:
                if cell and not in_sequence:
                    sides += 1
                    in_sequence = True
                elif not cell:
                    in_sequence = False
    
    return sides

def test_perimeter_num_sides():
    region = np.array([
        [0,0,0,1,1,0,0],
        [0,0,0,1,1,0,0],
        [0,1,1,1,0,0,0],
        [0,1,1,1,1,1,0],
        [0,1,1,0,1,1,0],
        [0,1,1,1,1,1,0]
    ], dtype=bool)
    
    for side in ['right', 'left', 'top', 'bottom']:
        print(f"\n{side.capitalize()} perimeter:")
        result = perimeter_side_cells(region, side)
        visualize_grid(result)
        num_sides = perimeter_num_sides(region, side)
        print(f"Number of sides: {num_sides}")

test_perimeter_num_sides()
```

Would you like me to explain how this works, or shall we use it to implement the complete solution for part 2?

In [ ]:
def perimeter_num_sides(region: np.ndarray, side: str) -> int:
    cells = perimeter_side_cells(region, side)
    sides = 0
    
    if side in ['top', 'bottom']:
        # Count sequences in each row
        for row in cells:
            in_sequence = False
            for cell in row:
                if cell and not in_sequence:
                    sides += 1
                    in_sequence = True
                elif not cell:
                    in_sequence = False
    else:  # left, right
        # Count sequences in each column
        for col in cells.T:  # transpose to iterate over columns
            in_sequence = False
            for cell in col:
                if cell and not in_sequence:
                    sides += 1
                    in_sequence = True
                elif not cell:
                    in_sequence = False
    
    return sides

def test_perimeter_num_sides():
    region = np.array([
        [0,0,0,1,1,0,0],
        [0,0,0,1,1,0,0],
        [0,1,1,1,0,0,0],
        [0,1,1,1,1,1,0],
        [0,1,1,0,1,1,0],
        [0,1,1,1,1,1,0]
    ], dtype=bool)
    
    for side in ['right', 'left', 'top', 'bottom']:
        print(f"\n{side.capitalize()} perimeter:")
        result = perimeter_side_cells(region, side)
        visualize_grid(result)
        num_sides = perimeter_num_sides(region, side)
        print(f"Number of sides: {num_sides}")

test_perimeter_num_sides()


Right perimeter:
....X..
....X..
...X...
.....X.
..X..X.
.....X.
Number of sides: 4

Left perimeter:
...X...
...X...
.X.....
.X.....
.X..X..
.X.....
Number of sides: 3

Top perimeter:
...XX..
.......
.XX....
....XX.
.......
...X...
Number of sides: 4

Bottom perimeter:
.......
....X..
.......
...X...
.......
.XXXXX.
Number of sides: 3


**AI Prompt**

now synthetize to function `compute_sides` with region as input

**AI Response**

Here's the implementation of `compute_sides` that sums up the sides from all four directions:

```python
def compute_sides(region: np.ndarray) -> int:
    total_sides = 0
    for side in ['top', 'bottom', 'left', 'right']:
        total_sides += perimeter_num_sides(region, side)
    return total_sides

def test_compute_sides():
    # Test case from the problem: E-shaped region
    e_shape = np.array([
        [1,1,1,1,1],
        [1,0,0,0,0],
        [1,1,1,1,1],
        [1,0,0,0,0],
        [1,1,1,1,1]
    ], dtype=bool)
    
    sides = compute_sides(e_shape)
    print("E-shape region:")
    visualize_grid(e_shape)
    print(f"Number of sides: {sides}")
    assert sides == 12, f"Expected 12 sides for E-shape, got {sides}"
    
    # Test simple square (4 sides)
    square = np.array([
        [1,1],
        [1,1]
    ], dtype=bool)
    sides = compute_sides(square)
    assert sides == 4, f"Expected 4 sides for square, got {sides}"
    
    print("Tests passed!")

test_compute_sides()
```

Would you like me to test it on more examples from the problem description, or shall we proceed with implementing the complete solution for part 2?

In [ ]:
def compute_sides(region: np.ndarray) -> int:
    total_sides = 0
    for side in ['top', 'bottom', 'left', 'right']:
        total_sides += perimeter_num_sides(region, side)
    return total_sides

def test_compute_sides():
    # Test case from the problem: E-shaped region
    e_shape = np.array([
        [1,1,1,1,1],
        [1,0,0,0,0],
        [1,1,1,1,1],
        [1,0,0,0,0],
        [1,1,1,1,1]
    ], dtype=bool)
    
    sides = compute_sides(e_shape)
    print("E-shape region:")
    visualize_grid(e_shape)
    print(f"Number of sides: {sides}")
    assert sides == 12, f"Expected 12 sides for E-shape, got {sides}"
    
    # Test simple square (4 sides)
    square = np.array([
        [1,1],
        [1,1]
    ], dtype=bool)
    sides = compute_sides(square)
    assert sides == 4, f"Expected 4 sides for square, got {sides}"
    
    print("Tests passed!")

test_compute_sides()

E-shape region:
XXXXX
X....
XXXXX
X....
XXXXX
Number of sides: 12
Tests passed!


**AI Prompt**

write updated function `total_price2` using `compute_sides`

**AI Response**

Here's the implementation of `total_price2` using the new sides calculation:

```python
def total_price2(regions: List[Tuple[str, np.ndarray]]) -> int:
    total = 0
    for plant, region in regions:
        cropped = crop_grid(region)
        area = compute_area(cropped)
        sides = compute_sides(cropped)
        total += area * sides
    return total

def test_total_price2():
    regions = find_regions(sample_grid)
    price = total_price2(regions)
    assert price == 1206, f"Expected 1206, got {price}"
    
    # Print details for verification
    for plant, region in regions:
        cropped = crop_grid(region)
        area = compute_area(cropped)
        sides = compute_sides(cropped)
        print(f"Plant {plant}: area={area} sides={sides} price={area*sides}")
    
    print("Test passed!")

test_total_price2()
```

Would you like to run this on the full input now?

In [ ]:
def total_price2(regions: List[Tuple[str, np.ndarray]]) -> int:
    total = 0
    for plant, region in regions:
        cropped = crop_grid(region)
        area = compute_area(cropped)
        sides = compute_sides(cropped)
        total += area * sides
    return total

def test_total_price2():
    regions = find_regions(sample_grid)
    price = total_price2(regions)
    assert price == 1206, f"Expected 1206, got {price}"
    
    # Print details for verification
    for plant, region in regions:
        cropped = crop_grid(region)
        area = compute_area(cropped)
        sides = compute_sides(cropped)
        print(f"Plant {plant}: area={area} sides={sides} price={area*sides}")
    
    print("Test passed!")

test_total_price2()

Plant R: area=12 sides=10 price=120
Plant I: area=4 sides=4 price=16
Plant C: area=14 sides=22 price=308
Plant F: area=10 sides=12 price=120
Plant V: area=13 sides=10 price=130
Plant J: area=11 sides=12 price=132
Plant C: area=1 sides=4 price=4
Plant E: area=13 sides=8 price=104
Plant I: area=14 sides=16 price=224
Plant M: area=5 sides=6 price=30
Plant S: area=3 sides=6 price=18
Test passed!


In [ ]:
def solve_part2():
    full_regions = find_regions(full_grid)
    return total_price2(full_regions)
solve_part2()

818286